<a href="https://colab.research.google.com/github/PabloDome/CosasUtilesDW/blob/main/ModeloPhi4_CicloHisteresis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from numba import jit

# programa para hacer estudios en AC con pulsos cuadrados, hace barridos en campo
Mostrar_Imagen = 0 # Aparece una ventana externa donde se ve la figura de los dominios
Mostrar_Mag_Emerg = 0 # Aparece una ventana externa con la figura de la magnetizacion

Guardar_imagenes = 1 # Guarda las imagenes de los dominios
Guardar_imagenes_txt = 0 # Guarda los datos de la imagen binaria en un txt

Guardar_Magnetizacion_txt = 0 # Guarda los datos de la imagen binaria en un txt
Calcular_area = 0 # Suma todos los pixeles blancos de la imagen binarizada (lo mismo que la magnetización pero multiplicado por el tamaño)
Calcular_perimetro = 0 # Suma los píxeles del contorno

#Direccion_Guardar = "/home/pc-mo/Documentos/RecreaciónPaperRugosidad" # Todo va a terminar en esta carpeta
#Direccion_Guardar = "/home/pablo/mnt/Users/Pablo/Desktop/Datos/Post-Doc/Modelo-Phi4/SimulacionAC/Imagenes" # Todo va a terminar en esta carpeta


# Datos del sistema
T = 0.0 # temperatura (ref = 0.01)
ampdis = 0.0 # Amplitud del desorden

MuestraSaturada = 1
SiDifMuestras = 1 # Crea muestras con diferentes patrones de desorden
e = 1.0 # sigma/delta = e (e va como A = 10 pJ/m = 10^{-11} J/m)
c = e # sigma*delta = c (c va como K = 1000 kJ/m3 = 10^6 J/m3)

Tam = 300 # Tamaño de la simulación - Tam = 300

#Detalles pared plana
ParedPlana = 0
L_plano = Tam/4

#Detalles circulos
RUnCirculo = 1 # Si pone o no un circulo
Rvarios = 0 # Si son varios circulos
Rd = 0
omega = 10
R_circ = 20 # Radio del circulo - 100
R_circ_CM_x = Tam*0.5
R_circ_CM_y = Tam*0.5

#Imagen Externa
ImagenExterna = 0 # Si es igual a 1 usa una imagen externa
DirImagen = '/home/pc-mo/Documentos/RecreaciónPaperRugosidad'
NombreIma = 'Recorte-Pulso110-2s-5'
ExtensioImagen = 'tif' # La extensión del archivo

#Detalles capilar
MuestraPeine = 0 # Si vale 1 te arma los peines
Ancho_capilar = Tam/10
dL = 20
L = Tam - 4*dL
L1 = 0.05*L
L2 = 0.15*L
L3 = 0.3*L
L4 = 0.4*L
R_circ_CM_x1 = dL+0.5*L1
R_circ_CM_x2 = 2*dL+L1+0.5*L2
R_circ_CM_x3 = 3*dL+L1+L2+0.5*L3
R_circ_CM_x4 = 4*dL+L1+L2+L3+0.5*L4
Y_min = Tam*0.5-Ancho_capilar*0.5
Y_max = Tam*0.5+Ancho_capilar*0.5
X_min = 15
X_max = Tam-10

dt_0 = 0.01 # tiempo maximo de una iteración en la simulación
dt = dt_0 # variación de tiempo, la maxima es dt = 0.1 después se pueden usar multiplos con dN

N_tot = 500 # estaba en 50000

h_max = 1.0 # Campo inicial
h_min = -1.0 # Campo final
h = 0.0

# Defino los parametros iniciales
Des_mat = (np.random.random((Tam, Tam)))*2.0 - 1.0
phi = -1*(np.ones((Tam,Tam)))
Muestra = np.ones((Tam,Tam))
forcePhi=-1*(np.ones((Tam,Tam)))
phi_print = np.zeros((Tam,Tam,3), dtype=np.uint8)
phi_Grey = phi_print


# Defino funciones
@jit(nopython=True)
def SelectorPeine(j):
    if j<dL:
        return 0
    elif j<dL+L1 and j>dL:
        return 1
    elif j>dL+L1 and j<2*dL+L1:
        return 0
    elif j>2*dL+L1 and j<2*dL+L1+L2:
        return 1
    elif j>2*dL+L1+L2 and j<3*dL+L1+L2:
        return 0
    elif j>3*dL+L1+L2 and j<3*dL+L1+L2+L3:
        return 1
    elif j>3*dL+L1+L2+L3 and j<4*dL+L1+L2+L3:
        return 0
    elif j>4*dL+L1+L2+L3 and j<4*dL+L1+L2+L3+L4:
        return 1
    else:
        return 0

@jit
def DefinirMuestra():

    #Defino un peine
    if MuestraPeine == 1:
        for i in range(Tam):
            for j in range(Tam):

                Muestra[i,j] = SelectorPeine(j)

                if i < X_min or i > X_max:
                    Muestra[i,j] = 1

                if i == 0 or j ==0 or i == Tam-1 or j == Tam-1:
                    Muestra[i,j] = 0


@jit(nopython=True)
def H_histeresis(tim):

    if (tim<=N_tot*0.5):
        h = h_max-2*tim*(h_max-h_min)/N_tot
    else :
        h = 2*h_min-h_max+2*tim*(h_max-h_min)/N_tot

    return h

@jit(nopython=True)
def signo(numero):
    if numero > 0:
        return 1
    elif numero < 0:
        return -1
    else:
        return 0

def rellenar_huecos(imagen):
    # Copiar la imagen de entrada
    imagen_rellena = imagen.copy()

    # Encontrar contornos en la imagen
    contornos, _ = cv2.findContours(imagen_rellena, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Rellenar los contornos encontrados
    for contorno in contornos:
        cv2.drawContours(imagen_rellena, [contorno], 0, 255, -1)

    return imagen_rellena

def eliminar_particulas(imagen, tamano_minimo):
    # Encuentra los contornos en la imagen binarizada
    contornos, _ = cv2.findContours(imagen.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Elimina las partículas que tienen un área menor que el tamaño mínimo
    for contorno in contornos:
        area = cv2.contourArea(contorno)
        if area < tamano_minimo:
            cv2.drawContours(imagen, [contorno], 0, 0, -1)

    return imagen

def Calcangulo(x,y):
    theta = 0

    if (x>0) and (y>=0) :
        theta = np.arctan(y/x)
    elif x==0 and (y>0) :
        theta = np.pi/2
    elif x<0 :
        theta = np.arctan(y/x) + np.pi
    elif x==0 and (y<0) :
        theta = 3*np.pi/2
    elif x>0 and y<0 :
        theta = np.arctan(y/x) + 2*np.pi

    return theta

#@jit(nopython=True) #Esto solo cambia valores a 1.0, por lo que si estan definidos en cero quedan asi
def selector(Val,i,j,R,Rx,Ry):

    x = (j-Rx)
    y = (i-Ry)

    if pow((i-Ry)*(i-Ry)+(j-Rx)*(j-Rx), 0.5)<R+Rd*np.sin(omega*Calcangulo(x,y)):
        return 1.0
    else:
        return Val

def ImagenParaPhi(Imagen):
    Imagen1 = 0*phi
    Imagen1 = cv2.resize(Imagen[:,:,0]/np.max(Imagen[:,:,0]),(Tam,Tam))

    for i in range(Tam):
        for j in range(Tam):
            if Imagen1[i,j]< 1:
                phi[i,j] = -1
            else:
                phi[i,j] = 1

def ini_matriz_circulos(phi):
    for i in range(Tam):
        for j in range(Tam):
            if Rvarios == 1:
                phi[i,j] = selector(phi[i,j],i,j,R_circ,R_circ_CM_x1,R_circ_CM_y)
                phi[i,j] = selector(phi[i,j],i,j,R_circ,R_circ_CM_x2,R_circ_CM_y)
                phi[i,j] = selector(phi[i,j],i,j,R_circ,R_circ_CM_x3,R_circ_CM_y)
                phi[i,j] = selector(phi[i,j],i,j,R_circ,R_circ_CM_x4,R_circ_CM_y)
            if RUnCirculo == 1:
                phi[i,j] = selector(phi[i,j],i,j,R_circ,R_circ_CM_x,R_circ_CM_y)

    if ImagenExterna == 1:
        Imagen = cv2.imread(DirImagen+'/'+NombreIma+'.'+ExtensioImagen)
        ImagenParaPhi(Imagen)

    phi = phi*Muestra
    return phi

@jit(nopython=True)
def ini_matriz_ParedPlana(phi):
    for i in range(Tam):
        for j in range(Tam):
            if i<L_plano:
                phi[i,j] = 1.0
    phi = phi*Muestra
    return phi

@jit(nopython=True)
def fuerza_elemento(phi, i, j, h, Des_mat):
    i_max = (i + 1)%Tam
    i_min = (i - 1 + Tam)%Tam
    j_max = (j + 1)%Tam
    j_min = (j - 1 + Tam)%Tam

    ev_phi = phi[i,j] + (c*(-4.0*phi[i,j] + phi[i_max,j] + phi[i_min,j] + phi[i,j_max] + phi[i,j_min]) + \
    e*(phi[i,j]*(1.0+ampdis*Des_mat[i,j]) - pow(phi[i,j], 3.0)) + h)*dt + \
    np.sqrt(T*dt/3.0)*np.random.uniform(-1, 1)

    return ev_phi

def euler_normal():
    i = 0
    while (i<Tam):
        j = 0
        while (j<Tam):
            if Muestra[i,j] == 1:
                phi[i,j] = fuerza_elemento(phi, i, j, h, Des_mat)
            elif Muestra[i,j] == 0:
                phi[i,j] = 0
            j += 1
        i += 1

@jit
def Imagen2(phi):
    for x in range(Tam):
        for y in range(Tam):
            if phi[x,y]>0:
                phi_print[x,y,0] = 255
                phi_print[x,y,1] = 0
                phi_print[x,y,2] = 0
            elif phi[x,y]<0:
                phi_print[x,y,0] = 0
                phi_print[x,y,1] = 0
                phi_print[x,y,2] = 255
            else:
                phi_print[x,y,0] = 0
                phi_print[x,y,1] = 0
                phi_print[x,y,2] = 0

def Imagen(phi):
    _, phi_print[:,:,0] = cv2.threshold(((phi + 1) / 2) * 255, 127, 255, cv2.THRESH_BINARY)
    phi_print[:,:,1] = 0*phi_print[:,:,0]
    phi_print[:,:,2] = 255-phi_print[:,:,0]

    for x in range(Tam):
        for y in range(Tam):
            if phi[x,y] == 0:
                phi_print[x,y,0] = 0
                phi_print[x,y,1] = 0
                phi_print[x,y,2] = 0


# Inicio simulación
DefinirMuestra()

if SiDifMuestras == 1:
    Des_mat = (np.random.random((Tam, Tam)))*2.0 - 1.0


if ParedPlana==1:
    phi = ini_matriz_ParedPlana(phi)
else:
    phi = ini_matriz_circulos(phi)

if MuestraSaturada==1:
    phi = 0*phi+1

M = np.zeros((N_tot,4))

tim = 0

while tim<N_tot :

    h = H_histeresis(tim) # (amplitud h, h_up, h_down, tim)

    k = 0
    Imagen2(phi)

    M[tim,0] = tim*dt
    M[tim,1] = h
    M[tim,2] = np.mean(phi)
    M[tim,3] = np.mean(phi_print[:,:,0]/255)*2-1

    euler_normal()

    tim = tim + 1


plt.plot(M[:,1],M[:,2]/np.max(M[:,2]))
plt.plot(M[:,1],M[:,3]/np.max(M[:,3]))
